In [1]:
import json, time, re, requests, pickle
import pandas as pd
import psycopg2 as pg2
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go


from sqlalchemy import create_engine
from psycopg2.extras import RealDictCursor, Json
from psycopg2.extensions import AsIs

%matplotlib inline
%run ../assets/sql_cred.py

/home/ubuntu/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def filename_format_log(file_path, 
                        logfile = '../assets/file_log.txt', 
                        now = round(time.time()), 
                        file_description = None): 
   
    try:
        ext = re.search('(?<!^)(?<!\.)\.(?!\.)', file_path).start() 
    except:
        raise NameError('Please enter a relative path with a file extension.') 
    
    stamp = re.search('(?<!^)(?<!\.)[A-z]+_[0-z]+(?=\.)', file_path).start()
    formatted_name = f'{file_path[:stamp]}{now}_{file_path[stamp:]}'  
    if not file_description:
        file_description = f'Saved at: {time.asctime(time.gmtime(now))}'
    with open(logfile, 'a+') as f:
        f.write(f'{formatted_name}: {file_description}\n')
    return formatted_name, now, file_description

In [3]:
engine = create_engine(ENGINE)

In [4]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    con = pg2.connect(host=IP_ADDRESS,
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

In [5]:
query = '''SELECT * FROM track_list;'''
response = execute_query(query, dict_cur=True)
track_df = pd.DataFrame(response)
track_df.set_index('track_id', inplace=True)
track_df.head()

,album_name,artist_name,lyrics,playlist_id,playlist_name,playlist_owner,track_name
track_id,,,,,,,
0h7TlF8gKb61aSm874s3cV,I Can't Tell You How Much It Hurts,moow,\n\nIf your needle is near\nNeedle is near\nYo...,37i9dQZF1DXarebqD2nAVg,Tender,spotify,You'r in My Head
6koowTu9pFHPEcZnACLKbK,Coming Home,Leon Bridges,\n\n[Verse 1]\nBrown skin girl on the other si...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Brown Skin Girl
1JkhKUXAoNivi87ipmV3rp,Back To Love (Deluxe Version),Anthony Hamilton,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Best of Me
51lPx6ZCSalL2kvSrDUyJc,The Search for Everything,John Mayer,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,You're Gonna Live Forever in Me
3vqlZUIT3rEmLaYKDBfb4Q,Songs In The Key Of Life,Stevie Wonder,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Isn't She Lovely


In [6]:
lyric_df = track_df[['lyrics']]

In [7]:
lyric_df = lyric_df.dropna(axis=0)

In [8]:
lyric_df.shape

(2276, 1)

In [9]:
lyric_df.head()

,lyrics
track_id,
0h7TlF8gKb61aSm874s3cV,\n\nIf your needle is near\nNeedle is near\nYo...
6koowTu9pFHPEcZnACLKbK,\n\n[Verse 1]\nBrown skin girl on the other si...
1JkhKUXAoNivi87ipmV3rp,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ..."
51lPx6ZCSalL2kvSrDUyJc,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...
3vqlZUIT3rEmLaYKDBfb4Q,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...


In [10]:
def clean_lyrics(lyrics, keep_tags=False, keep_nl=False):

    text = lyrics.lower()

    if not keep_tags:
        text = re.sub('(\[.+\])', '', text)

    text = re.sub('[^a-z\s]', '', text)

    text = text.strip()
    text = re.sub('\n\n', ' ', text) 

    text = text.strip()

    if keep_nl:
        text = re.sub('\n', ' \n ', text)
    else:
        text = re.sub('\n', ' ', text)


    return text

In [11]:
lyric_df['clean_lyrics'] = lyric_df.lyrics.map(lambda x: clean_lyrics(x, keep_tags=False, keep_nl=True))

In [12]:
lyric_df.head()

,lyrics,clean_lyrics
track_id,,
0h7TlF8gKb61aSm874s3cV,\n\nIf your needle is near\nNeedle is near\nYo...,if your needle is near \n needle is near \n yo...
6koowTu9pFHPEcZnACLKbK,\n\n[Verse 1]\nBrown skin girl on the other si...,brown skin girl on the other side of the room ...
1JkhKUXAoNivi87ipmV3rp,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",its simple i love it \n having you near me hav...
51lPx6ZCSalL2kvSrDUyJc,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,a great big bang and dinosaurs \n fiery rainin...
3vqlZUIT3rEmLaYKDBfb4Q,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,isnt she lovely \n isnt she wonderful \n isnt ...


In [13]:
not_eng = lyric_df[lyric_df['clean_lyrics'].str.contains('(\sdel\s)|(\sque\s)|(\svous\s)|(\setre\s)')].index.tolist()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [14]:
lyric_df = lyric_df.drop(index=not_eng, axis=0)

In [15]:
not_lyrics = lyric_df[lyric_df['clean_lyrics'].str.contains('lyrics for this song have yet')].index.tolist()

In [16]:
lyric_df = lyric_df.drop(index=not_lyrics, axis=0)

In [17]:
long_gap = lyric_df[lyric_df['clean_lyrics'].str.contains(r'(\s{6,})')].index.tolist()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

This pattern has match groups. To actually get the groups, use str.extract.



In [19]:
lyric_df = lyric_df.drop(index=long_gap, axis=0)

In [20]:
lyric_df = lyric_df.drop_duplicates(subset=['clean_lyrics'])

In [21]:
lyric_df.describe()

,lyrics,clean_lyrics
count,1787,1787
unique,1787,1787
top,\n\n[Intro]\nOh yeah\n\n[Verse 1]\nIt's been o...,i guess its funnier from where your standing \...
freq,1,1


In [22]:
corpus = []
words_track = []    
lines_track = []

for _, track in lyric_df[['clean_lyrics']].iterrows():
    #words in full set
    lyrics = track[0]
    lyrics_spaced = re.sub(r'( +)', ' ', lyrics)
    lyrics_split = lyrics_spaced.split(' ')
    corpus.extend(lyrics_split)
    
    #lines in track
    lines_split = lyrics_spaced.split('\n')
    lines_track.append(lines_split)
    
    #words in track w/o \n 
    lyrics_spaced = re.sub(r'\n ', '', lyrics_spaced)
    lyrics_split = lyrics_spaced.split(' ')
    words_track.append(lyrics_split)

In [23]:
vocab = set(list(corpus))

In [24]:
len(vocab)

12977

In [25]:
count_words_track = []
for i in words_track:
    count_words_track.append(len(i))

In [27]:
lyric_df['total_words_track'] = count_words_track

In [28]:
unique_words_track = []
for track in words_track:
    unique_words_track.append(set(track))

In [29]:
count_unique_words_track = []
for track in unique_words_track:
    count_unique_words_track.append(len(track))

In [30]:
lyric_df['unique_words_track'] = count_unique_words_track

In [31]:
np.mean(count_words_track)

284.1980973698937

In [32]:
len_words_track = []
for i in words_track:
    len_words = []
    for word in i:
        len_words.append(len(word))
    len_words_track.append(np.mean(len_words))

In [33]:
np.mean(len_words_track)

3.7477948763594697

In [37]:
count_lines_track = []
for i in lines_track:
    count_lines_track.append(len(i))

In [38]:
lyric_df['total_lines_track'] = count_lines_track

In [35]:
unique_lines_track = []
for track in lines_track:
    unique_lines_track.append(set(track))

In [36]:
count_unique_lines_track = []
for track in unique_lines_track:
    count_unique_lines_track.append(len(track))

In [39]:
lyric_df['unique_lines_track'] = count_unique_lines_track

In [40]:
lyric_df.head()

,lyrics,clean_lyrics,total_words_track,unique_words_track,total_lines_track,unique_lines_track
track_id,,,,,,
0h7TlF8gKb61aSm874s3cV,\n\nIf your needle is near\nNeedle is near\nYo...,if your needle is near \n needle is near \n yo...,57,20,14,8
6koowTu9pFHPEcZnACLKbK,\n\n[Verse 1]\nBrown skin girl on the other si...,brown skin girl on the other side of the room ...,132,52,24,13
1JkhKUXAoNivi87ipmV3rp,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",its simple i love it \n having you near me hav...,151,63,29,21
51lPx6ZCSalL2kvSrDUyJc,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,a great big bang and dinosaurs \n fiery rainin...,126,76,20,18
3vqlZUIT3rEmLaYKDBfb4Q,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,isnt she lovely \n isnt she wonderful \n isnt ...,108,66,21,20


In [41]:
np.mean(count_lines_track)

40.61723559037493

In [42]:
len_word = 0
for word in vocab:
    len_word += len(word)

In [43]:
mean_len_word = len_word / len(vocab)
mean_len_word

6.466286506896817

In [44]:
lines_track[:2]

[['if your needle is near ',
  ' needle is near ',
  ' you can take my blood ',
  ' oh i saved it for you ',
  ' hundreds of drops ',
  ' running red needle is near ',
  ' you can take my blood ',
  ' oh i saved it for you ',
  ' hundreds of drops ',
  ' running red ',
  ' needle is near ',
  ' oh i saved it for you ',
  ' hundreds of drops ',
  ' running red'],
 ['brown skin girl on the other side of the room ',
  ' brown skin girl staring with her brown eyes ',
  ' ooh baby dont you know youre a cutie pie ',
  ' princess little honey with the polkadot dress on ',
  ' rubylipped lady whose name i dont know ',
  ' let me tell you darling ',
  ' ooh ooh ooh ',
  ' ooh whooooh ',
  ' ooh ooh ooh ',
  ' ooh whooooh ok ',
  ' brown skin girl with the white pearls round her neck ',
  ' brown skin girl let me hold you close under the white moon ',
  ' cause baby im ready and you know that im waiting on you ',
  ' princess little honey with the polkadot dress on ',
  ' rubylipped lady whose n

In [45]:
mean_words_line = []
for track in lines_track:
    words_line = []
    for line in track:
        words_line.append(len(line.split(' ')))
    mean_words_line.append(np.around(np.mean(words_line),1))

In [47]:
lyric_df['mean_words_line'] = mean_words_line

In [50]:
mean_unique_words_line = []
for track in lines_track:
    unique_words_line = []
    for line in track:
        unique_words_line.append(len(set(line.split(' '))))
    mean_unique_words_line.append(np.around(np.mean(unique_words_line),1))

In [52]:
lyric_df['mean_unique_words_line'] = mean_unique_words_line

In [54]:
plt.hist(count_lines_track, range=(0, 125));

NameError: name 'plt' is not defined

In [ ]:
plt.hist(count_words_track, range=(0, 1000));

In [ ]:
plt.hist(count_words_line, range=(0,20));

In [ ]:
plt.hist(len_words_track);

In [ ]:
trace = go.histogram(x=words_line)
data = trace
py.iplot([trace])

In [ ]:
# formatted_name, now, file_description= filename_format_log(file_path = '../assets/clean_lyrics.csv')

# lyric_df.to_csv(formatted_name, index=False)

In [ ]:
eda_df = pd.DataFrame(columns=['id', 'line', 'line_num', 'tag'])

for idx, track in lyric_df[['lyrics']].iterrows():
    track = re.sub('(\[[^\]]+\]\\n\\n)', '', track[0]) #removes tags without lyrics
    track = re.sub('\[[^\[]+(:[^\]]+)\]', '', track) #removes all text after :
    track = re.sub('\[[^\[]+(–[^\]]+)|\[[^\[]+(-\s[^\]]+)\]', '', track) #removes all after –
    track = re.sub('\[[^\[]+(\(([^\]]+))\]', '', track) #removes all after ()
    track = re.sub('(^|)(\[Ad-Libs\])|(\[Keisha\])|(\[Shane Powers\])', '', track) #removes artist and non-standard tags
    track = re.sub('(^|)(\[Break\])|(\[Vocals / Lyrics By\])|(\[Skit\])', '', track) #removes artist and non-standard tags
    
    tags = re.findall('(\[[^\]]+\])', track) #finds all tags
    track = re.sub('(\[[^\]]+\])', '', track) #removes all tags
              
    track = track.strip().split('\n\n') 
    track_split = [verse.strip().split('\n') for verse in track] 
    
    v_count = len(track_split)

    for i in range(v_count):
        if len(tags) == v_count:
            tag = tags[i]
        else:
            tag = f'[verse {(i+1)}]'
        
        for j, line in enumerate(track_split[i]):
            index = 1
            
            row = {
            'id': idx,
            'line':line,
            'line_num':(j+1),
            'tag': tag.lower()
            }
            
            eda_df = eda_df.append(row, ignore_index=True)

In [ ]:
eda_df

In [ ]:
# formatted_name, now, file_description = filename_format_log(file_path ='../assets/eda_df.csv')
# eda_df.to_csv(formatted_name, index=False)

In [ ]:
# eda_df = pd.read_csv('../assets/1548892595_eda_df.csv')

In [ ]:
eda_df.tag.value_counts()[eda_df.tag.value_counts() < 10]

In [ ]:
drop = eda_df[eda_df['id'] == '4TYZXfu6VeblQMK2TwbDte'].index.tolist()

In [ ]:
eda_df = eda_df.drop(drop, axis=0)

In [ ]:
eda_df[eda_df.loc[:,'tag'] == '[verse 55]']

In [ ]:
lyric_df.loc['6xYd4zCVeSp80Un2Rl9wDs',:]['lyrics']

In [ ]:
off_verse = eda_df[eda_df.id =='6xYd4zCVeSp80Un2Rl9wDs'].index.tolist()

In [ ]:
eda_df.tag.loc[off_verse] = eda_df.tag.loc[off_verse].map(lambda x: '[verse 1]')

In [ ]:
eda_df.loc[off_verse,'line_num'] = list(range(1,63))

In [ ]:
eda_df.tag.value_counts()

In [ ]:
eda_df[eda_df.loc[:,'tag'] == '[verse 28]']

In [ ]:
leagues = eda_df[eda_df.id =='2za6PlkAQAMMu0VFsfBoHC'].index.tolist()

In [ ]:
eda_df = eda_df.drop(index=leagues, axis=0)

In [ ]:
eda_df.describe()

In [ ]:
eda_df[eda_df.id == '3yPMLvxeEor7rbDLDseJFt']

In [ ]:
tracks = eda_df[eda_df.id == '3yPMLvxeEor7rbDLDseJFt'].index.tolist()

In [ ]:
eda_df = eda_df.drop(index=tracks, axis=0)

In [ ]:
eda_df.describe()

In [ ]:
eda_df.tag.value_counts()

In [ ]:
eda_df[eda_df.tag.str.contains('\n')]

In [ ]:
lyric_df.loc['1Ob94QesoxHSBrypyBbdfd'].lyrics = "\n\n[Verse 1]\nWe've been here before, surrounded in the cold\nYou take me to places I've never known\nAnd you push me to places I'll never go\nI would die for you\nTell me the truth\nAm I all that you would need?\nIf we're keeping it a hundred, you're all that I need for me\nFrom the jump until forever from now, I would ride for you\n\n[Chorus]\nBut you got me shot down by love\nYou got my heart now\nWhy won't you stop now?\nOh I've been knocked down by you\nYou got my heart now\nWhy won't you stop now?\n\n[Verse 2]\nI've been through it whole\nI've been through the worst\nBut I never knew how much our love could hurt\nOver my family I put you first\nWriting out my feelings is the only thing that work\nI don't hear from the friends I thought were mine too\nBut I hold on to the poems I would write you\nHappy seventeen, I saw us two and you saw three\nI guess our lines are structured like a haiku\n\n[Chorus 2]\nYou got me shot down by love\nAnd you got my heart now\nWhy won't you stop now?\nOh I've been knocked down by you\nAnd you got my heart now\nWhy won't you stop now?\n\n[Bridge]\nKnocked down, knocked down\nKnocked down, knocked down\nKnocked down\nKnocked down\n\n[Chorus]\nI've been shot down by love\nYou got my heart now\nWhy won't you stop now?\nI've been knocked down by you\nAnd you got my heart now\nWhy won't you stop now?\n\n"

In [ ]:
bad_tag = eda_df.tag[eda_df.tag.str.contains('\n')].index.tolist()

In [ ]:
eda_df = eda_df.drop(index=bad_tag,axis=0)

In [ ]:
eda_df.tag.value_counts()

In [ ]:
trace = go.histogarm(x=eda_df.tag)
data = trace
py.iplot([trace])